In [8]:
import pandas as pd
import os
import os
from dotenv import load_dotenv
load_dotenv()

True

## Load Data

In [9]:
path = os.getenv("path_to_data")
path_master = os.getenv("path_to_master")

In [22]:
df = pd.read_csv(f"{path_master}/messungen_2024.csv", delimiter =";").T
df.columns= df.iloc[0]
df = df.iloc[1:].copy()
df.columns = ["zeit", "ecoli", "entro", "pos_neg"]
df.index.rename("DATUM", inplace = True)

In [11]:
for col in ["ecoli", "entro"]:
    df[col] = df[col].str.replace(">800", "8000")
    df[col] = df[col].str.replace(">80", "8000")
    df[col] = df[col].str.replace(">", "").astype(float)

In [12]:
df.reset_index(inplace = True)

In [13]:
month_map = {
    'Jan': '01.',
    'Febr': '02.',
    'Mär': '03.',
    'Apr': '04.',
    'Mai': '05.',
    'Jun': '06.',
    'Jul': '07.',
    'Aug': '08.',
    'Sep': '09.',
    'Okt': '10.',
    'Nov': '11.',
    'Dez': '12.'
}

for key, value in month_map.items():
    df["DATUM"] = df["DATUM"].str.replace(key, value)

In [14]:
def fill_year(date):
    if date[-2:] != "24":
        return date + " 24"
    else:
        return date

In [15]:
def fix_date(date):
    date = date.split(" ")
    date = "20" + date[-1] + "-" + date[1][:-1] + "-" + date[0][:-1]
    return date

In [16]:
def add_time(date, time):
    time = time.apply(lambda x: x.split(":")[0])
    date = date + " " + time + ":00:00"
    return date

In [17]:
df["DATUM"] = df["DATUM"].apply(fill_year).apply(fix_date)
df["DATUM"] = add_time(df["DATUM"], df["zeit"])

In [18]:
df["DATUM"] = pd.to_datetime(df["DATUM"])

In [19]:
df["DATUM"] = df["DATUM"].dt.round('h')

In [20]:
df.set_index("DATUM", inplace = True)

In [ ]:
df.to_csv(f"{path_master}/messungen_clean_2024.csv")